# Program Topic Modelling LDA dan LSA 

Latent Dirichlet Allocation (LDA) adalah model probabilistik generatif dari koleksi data diskrit seperti korpus teks. Ide dasarnya adalah bahwa dokumen direpresentasikan sebagai campuran acak atas topik laten (tidak terlihat).

LDA merupakan model Bayesian hirarki tiga tingkat, di mana setiap item koleksi dimodelkan sebagai campuran terbatas atas serangkaian set topik. Setiap topik dimodelkan sebagai campuran tak terbatas melalui set yang mendasari probabilitas topik. Dalam konteks pembuatan model teks, probabilitas topik memberikan representasi eksplisit dari sebuah dokumen.

Algoritma LSA (Latent Semantic Analysis) adalah salah satu algoritma yang dapat digunakan untuk menganalisa hubungan antara sebuah frase/kalimat dengan sekumpulan dokumen. Contoh yang dibahas kali ini adalah mengenai penentuan urutan peringkat data berdasarkan query yang digunakan. 

<h3>Install Library</h3>

<p>Dibawah ini dicantumkan beberapa library yang dibutuhkan untuk menlakukan processing menggunakan LSA dengan jupyter notebook, anda juga bisa melakukan instalasi ini dengan menggunakan Command Prompt untuk pengguna Windows atau Terminal untuk pengguna Linux</p>

- pip install numpy
- pip install sklearn
- pip install pandas
- pip install matplotlib
- pip install seaborn
- pip install nltk

<h3>Import Library</h3>

<p>Untuk library yang digunakan diantaranya ada numpy, pandas, matplotlib, seaborn, nltk, dan sklearn, library ini umum digunakan pada data processing</p>

In [4]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer


#stop-words
stop_words=set(nltk.corpus.stopwords.words('indonesian'))

<h3>Instalasi Library Tambahan</h3>
<p>Di bawah ini ada library tambahan yang harus di-install untuk memproses kata-kata yang diolah</p>

In [ ]:
nltk.download('corpus')
nltk.download('stopwords')
nltk.download('punkt')

<h3>Import dokumen</h3>
<p>Import dokumen yang sudah dicrawling dengan crawler, bisa menggunakan referensi dari web ini atau bisa menggunakan referensi kode dari website lain</p>

In [5]:
df=pd.read_csv(r'abstrak_pta_new.csv')

<h2>Tampilan dari 10 data yang diproses</h2>

In [6]:
df.head(10)

,judul,abstraksi
0,KARAKTERISTIK KONDISI OSEANOGRAFI DAN DISTRIBU...,Laut Jawa (Wilayah Pengelolaan Perikanan/ WPP ...
1,A STUDY OF METADISCOURSE MARKERS IN GRETA THUN...,Penelitian ini bertujuan untuk membahas jenis ...
2,Jaringan Sosial Pengusaha Kerajinan Limbah Kay...,"Ana Febrianti, 160521100004 Program Studi Sosi..."
3,Analisis strategi pengelolaan Pantai Syariah P...,"Di Indonesia, potensi pengembangan Industri pa..."
4,MEMAKNAI GAYA HIDUP PENIKMAT KOPI KHAS INDONES...,Siti Maghfuro 160521100023 Program Studi Sosio...
5,Pertukaran Sosial Dalam Tradisi Sape Tok-tok,Abstrak – Farida Yulistiana Aji. \n16052110000...
6,Public Awareness Masyarakat Madura Tentang Kon...,Penelitian ini dengan judul “Public Awareness ...
7,HUBUNGAN ANTARA PROKRASTINASI AKADEMIK DENGAN ...,Penelitian ini bertujuan untuk mengetahui hubu...
8,Evaluasi Tingkat Kelelahan Kerja Perawat Ruang...,Rumah sakit merupakan salah satu instansi di b...
9,Pengaruh Biochar Sekam Padi dan Pupuk Kandang ...,Biochar merupakan padatan yang kaya kandungan ...


<h3>Pembersihan dokumen</h3>

<p>Pembersihan dokumen diperlukan agar dalam proses TF/IDF tidak ada simbol-simbol yang ikut masuk ke dalam proses tersebut yang dapat mengakibatkan dokumen menjadi kurang otentik</p>

In [7]:
def clean_text(headline):
    le=WordNetLemmatizer()
    word_tokens=word_tokenize(headline)
    tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
    cleaned_text=" ".join(tokens)
    return cleaned_text

In [8]:
# time taking
df['abstrak_cleaned']=df['abstraksi'].apply(clean_text)

<h3>Perbandingan data yang belum dan sudah dibersihkan</h3>

In [9]:
df.head()

,judul,abstraksi,abstrak_cleaned
0,KARAKTERISTIK KONDISI OSEANOGRAFI DAN DISTRIBU...,Laut Jawa (Wilayah Pengelolaan Perikanan/ WPP ...,Laut Jawa Wilayah Pengelolaan Perikanan/ perai...
1,A STUDY OF METADISCOURSE MARKERS IN GRETA THUN...,Penelitian ini bertujuan untuk membahas jenis ...,Penelitian bertujuan membahas jenis tanda meta...
2,Jaringan Sosial Pengusaha Kerajinan Limbah Kay...,"Ana Febrianti, 160521100004 Program Studi Sosi...",Febrianti 160521100004 Program Studi Sosiologi...
3,Analisis strategi pengelolaan Pantai Syariah P...,"Di Indonesia, potensi pengembangan Industri pa...",Indonesia potensi pengembangan Industri pariwi...
4,MEMAKNAI GAYA HIDUP PENIKMAT KOPI KHAS INDONES...,Siti Maghfuro 160521100023 Program Studi Sosio...,Siti Maghfuro 160521100023 Program Studi Sosio...


<p>Drop kolom abstraksi</p>

In [10]:
df.drop(['abstraksi'],axis=1,inplace=True)

<p>Data frame kolom abstrak_cleaned</p>

In [12]:
df['abstrak_cleaned'][0]

'Laut Jawa Wilayah Pengelolaan Perikanan/ perairan potensial habitat kelompok jenis ikan pelagis ekonomis Indonesia total estimasi potensi 981,680 ton/tahun Target penangkapan ikan Laut Jawa ikan pelagis Penangkapan ikan Laut Jawa malam bantuan cahaya lampu sasaran ikan berfototaksis positif Tujuan penelitian menganalisa distribusi kapal light fishing karakteristik parameter oseanografi kapal light fishing perairan Laut Jawa Penelitian data posisi kapal light fishing data harian 2016-2019 parameter oseanografi Salinitas diunduh CMEMS suhu Klorofil-a diunduh AQUA-Modis Level3 Hasil penelitian menunjukan distribusi dideteksi musim timur terendah musim barat Kondisi parameter oseanografi Laut Jawa suhu berkisar 27-34,5°C Klorofil-a berkisar 0,1-5 mg/m3 Salinitas berkisar 30-34,6 berkisar 0,44-0,77 meter Hasil ekstraksi data posisi kapal parameter oseanografi diindikasikan daerah light fishing kondisi oseanografi suhu 28-31°C klorofil-a berkisar 0,2-0,5 mg/m3 salinitas berkisar 32-33 muka 

## Term Frequency

<p>Data di atas adalah data yang sudah diproses menggunakan TF-IDF untuk menentukan Term Frequency tiap topik. TF-IDF (Term Frequency - Inverse Document Frequency) adalah algoritma praktis yang menggunakan frekuensi kata untuk menentukan seberapa relevan kata-kata itu dengan dokumen tertentu. Ini adalah pendekatan yang relatif sederhana namun intuitif untuk pembobotan kata, memungkinkannya bertindak sebagai titik awal yang bagus untuk berbagai tugas.</p>
<h3>Rumus TF-IDF</h3>

$$
\operatorname{tf}(t, d)=\frac{f_{t, d}}{\sum_{t^{\prime} \in d} f_{t^{\prime}, d}}
$$

<p></p>

In [13]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000) # to play with. min_df,max_df,max_features etc...

In [14]:
vect_text=vect.fit_transform(df['abstrak_cleaned'])

C:\Users\WINDOWS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [28]:
print(vect_text.shape)
#print(vect_text)
type(vect_text)
df=pd.DataFrame(vect_text.toarray())
print(df)
idf=vect.idf_

(60, 1000)
         0         1         2         3         4         5         6    \
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7   0.063018  0.066479  0.000000  0.000000  0.055319  0.000000  0.000000   
8   0.245848  0.000000  0.049532  0.000000  0.215815  0.091960  0.000000   
9   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
10  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
11  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1

In [16]:
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['jaringan'])
print(dd['strategi'])  # police is most common and forecast is least common among the news headlines.

penelitian zakat
3.501435951739211
3.164963715117998


C:\Users\WINDOWS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Proses LSA

<p>LSA  adalah metode yang memungkinkan kita mengekstrak topik dari dokumen dengan mengubah teksnya menjadi matriks topik-kata dan topik-dokumen. Prosedur untuk LSA relatif mudah: Ubah korpus teks menjadi matriks istilah dokumen. Menerapkan dekomposisi nilai singular terpotong.</p>

$A_{m n}=U_{m m} x S_{m n} x V_{n n}^{T}$

<p>Matriks U = baris merepresentasikan vektor pada topic dokumen</p>
<p>Matriks V = Garis ini merepresentasikan vektor istilah yang dinyatakan pada topik</p>
<p>Matriks S = Matriks diagonal yang memiliki elemen-elemen diagonal yang digunakan sebagai nilai singular A</p>

<p>tiap baris pada matriks U merupakan representasi vektor yang terdapat pada dokumen yang sesuai, untuk melakukannya dapat menggunakan library Sklearn yang bernama TruncatedSVD untuk menimplementasikan LSA</p>

In [17]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 1.06153173e-01 -4.04889332e-02  6.12160674e-02  4.58312267e-02
   1.57819285e-02 -7.31249152e-02  1.07934847e-01  1.91987176e-01
   3.21317895e-01 -3.86411102e-01]
 [ 7.34678089e-02 -9.83625445e-03 -7.48854394e-03 -3.21229303e-04
  -8.35530608e-03  1.66583220e-02  7.45085438e-03  1.11982584e-02
  -3.34062654e-02 -4.44699267e-02]
 [ 2.50698413e-01 -9.54330482e-02 -8.22691943e-02 -1.64814286e-01
  -2.00113087e-01  1.35749630e-02  2.16535887e-02 -3.43899922e-02
  -1.08412557e-01 -5.37433120e-02]
 [ 2.87867126e-01 -9.78614435e-02 -1.16258225e-01 -1.45714700e-01
   2.43208959e-01 -1.08259114e-01  4.75555474e-02  7.24248880e-02
   4.55415943e-01  4.29230224e-01]
 [ 1.95567729e-01 -7.17802134e-02 -4.80570276e-02 -8.77346362e-02
  -9.35055215e-02 -1.87195481e-02  8.90404611e-03  5.98101282e-03
  -3.41380599e-03 -2.29764579e-02]
 [ 2.28456957e-01 -1.21555458e-01 -1.02378651e-01 -2.46547199e-01
  -2.57017297e-01 -6.75915419e-02 -3.52874892e-02 -2.18577397e-02
  -1.10082802e-01 -7.23421606e-02

In [18]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  10.615317336988117
Topic  1  :  -4.048893315556796
Topic  2  :  6.121606740094335
Topic  3  :  4.583122674813532
Topic  4  :  1.5781928522040856
Topic  5  :  -7.3124915175911145
Topic  6  :  10.793484725991226
Topic  7  :  19.198717598713902
Topic  8  :  32.13178946757749
Topic  9  :  -38.64111019991006


In [19]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 1000)
[[ 3.68838453e-02  1.73717411e-02  6.32473900e-03 ...  6.18604097e-03
   3.30547197e-02  8.74155242e-02]
 [ 2.13805687e-05 -9.65632006e-03 -2.43102682e-03 ... -2.76681134e-03
  -1.63568072e-02 -6.33835619e-02]
 [ 7.48893280e-03  3.47604581e-03  4.42805804e-04 ... -2.04348593e-03
  -1.67151577e-02 -6.23375515e-02]
 ...
 [-2.37082050e-02 -4.46456260e-03 -2.73535973e-06 ...  4.32974349e-04
  -6.98248468e-03 -5.92131285e-03]
 [ 2.31196651e-02  1.82491007e-02  4.23603579e-05 ... -4.76079455e-03
  -5.80949640e-02  9.76001544e-03]
 [-1.75759290e-02 -1.50289603e-03  2.40960725e-03 ... -2.55675632e-03
  -6.19868442e-02  3.29305015e-02]]


<h3>Hasil TruncatedSVD</h3>

berikut adalah contoh 10 kata penting ditiap topik yang diproses

In [20]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
penelitian siswa kerja halal data masyarakat sosial wisata tradisi kualitas 

Topic 1: 
siswa pembelajaran guru miskonsepsi perangkat kemampuan pemecahan persentase belajar rendah 

Topic 2: 
pemangkasan pucuk tanaman bobot basah perlakuan kandang pupuk umur kering 

Topic 3: 
kerja variabel garam kualitas shift perusahaan karyawan tenaga kepuasan signifikan 

Topic 4: 
halal wisata literate literasi pariwisata chain sumenep value sertifikasi produk 

Topic 5: 
akademik prokrastinasi mahasiswa skala burnout coping anak regulasi psikologi korelasi 

Topic 6: 
perusahaan ratio bopo faktor harga pembelian signifikan produk bank berpengaruh 

Topic 7: 
risiko tsunami umroh haji tabung skenario pembiayaan manajemen foundation yulianto 

Topic 8: 
wisata pariwisata pantai ikan pulau klorofil syariah oseanografi perairan laut 

Topic 9: 
wisata pariwisata syariah pemangkasan pucuk bank anak risiko religi layanan 



## LDA (Latent Dirichlet Allocation)
Latent Dirichlet Allocation (LDA) adalah teknik pemodelan topik yang populer untuk mengekstrak topik dari korpus tertentu. Istilah laten menyampaikan sesuatu yang sudah ada tetapi belum berkembang. 

In [21]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
# n_components is the number of topics

In [22]:
lda_top=lda_model.fit_transform(vect_text)

In [23]:
print(lda_top.shape)  # (no_of_doc,no_of_topics)
print(lda_top)

(60, 10)
[[0.01500002 0.01499895 0.01499885 0.01499934 0.0149992  0.01499925
  0.0149992  0.86500721 0.01499897 0.01499901]
 [0.01972556 0.01972723 0.01972538 0.01972536 0.01972535 0.82246959
  0.01972524 0.01972541 0.01972565 0.01972524]
 [0.01671969 0.01671994 0.01671988 0.01672005 0.84951952 0.01672024
  0.01671969 0.01671996 0.01672026 0.01672077]
 [0.01371976 0.01371997 0.01371978 0.01371974 0.01371978 0.0137199
  0.01371985 0.01371981 0.87652157 0.01371983]
 [0.01625076 0.01625094 0.85373962 0.01625096 0.01625113 0.016251
  0.01625091 0.01625096 0.016251   0.01625272]
 [0.01873823 0.01873851 0.01873854 0.01873828 0.01873855 0.01873864
  0.01873842 0.01873867 0.01873889 0.83135327]
 [0.01819806 0.01819762 0.01819788 0.01819735 0.01819763 0.0181971
  0.0181977  0.0181977  0.83622142 0.01819754]
 [0.01787393 0.01787349 0.01787406 0.83913471 0.01787413 0.01787404
  0.01787436 0.01787383 0.0178735  0.01787394]
 [0.01771126 0.01771152 0.01771181 0.01771143 0.0177115  0.01771161
  0.017

In [24]:
sum=0
for i in lda_top[0]:
  sum=sum+i
print(sum)

1.0000000000000002


In [25]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  1.5000020109940448 %
Topic  1 :  1.4998948522953661 %
Topic  2 :  1.499884581428648 %
Topic  3 :  1.4999342674692702 %
Topic  4 :  1.4999198059980519 %
Topic  5 :  1.499925398151928 %
Topic  6 :  1.4999200935279642 %
Topic  7 :  86.50072103441734 %
Topic  8 :  1.499897235121079 %
Topic  9 :  1.4999007205963226 %


In [26]:
print(lda_model.components_)
print(lda_model.components_.shape)  # (no_of_topics*no_of_words)

[[0.87031114 0.83571135 0.81044021 ... 0.92735631 0.75631235 0.84881569]
 [0.91109752 0.87067841 0.62901977 ... 0.82639955 0.85778536 1.05082305]
 [0.68888381 0.83642775 0.88243714 ... 0.93366613 0.77714325 0.80835197]
 ...
 [0.75684586 0.91820295 0.85334306 ... 0.79899825 0.67643385 0.81364844]
 [0.9953409  0.76222534 0.82673198 ... 0.88485638 0.79560402 0.8123469 ]
 [0.75327413 0.76478821 0.93785777 ... 0.87075443 0.69845767 0.72556051]]
(10, 1000)


In [27]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
berkearifan sekunder tipe kecerdasan signifikan marketing pesisir aktor pekerja berprestasi 

Topic 1: 
pandangan ketapang risiko tipe laki rendah tsunami batik ziarah konsentrasi 

Topic 2: 
aspek sape kopi pola tourism penggunaan sentra mesin sampling alat 

Topic 3: 
stemming mahasiswa silika didalam menyimpulkan kesehatan karang makam fungsi tepung 

Topic 4: 
penelitian moment literate kabupaten kewirausahaan pemeriksaan gerbang menganalisis 00 program 

Topic 5: 
simultan tersendiri dianis rajungan rumah perairan fosfat kertasada sapi kebudayaan 

Topic 6: 
perlakuan masuk ton pokok manajemen media pembelian kholil peneliti sapi 

Topic 7: 
berkisar kepribadian layanan kebun sensor religi saham penilaian mengunakan jenuh 

Topic 8: 
singkong halal sumberdaya shift kecerahan time massa manten menikmati pertemuan 

Topic 9: 
pembelajaran subjek rendah aktual stasiun berkah jati pelagis 12 korelasi 

